# <font color=#0000FF>每10分鐘雲資料重取樣為每15分鐘一筆資料，此外將low為NaN的值用mid來填補、將mid為NaN的值用hig來填補</font>

In [1]:
#將十分鐘一筆資料重取樣為15分鐘一筆。ffill方法用來填補缺失值，確保資料能夠被正確重取樣。
#然後使用isin方法檢查df2(15m)中的日期是否在df(10m)中存在，如果存在則將資料複製到df2中。最後，使用interpolate方法對df2進行線性插值。
import pandas as pd
import numpy as np

# 讀取資料
df = pd.read_csv(f'./cloud_cover/new_fst.v38.csv')

# 將date_time欄位轉換成datetime格式
df['fst_time'] = pd.to_datetime(df['fst_time'])
print("--------------------初始")
print(df.head(20))

# 做填充
df['mid'].fillna(df['hig'], inplace=True)
df['low'].fillna(df['mid'], inplace=True)
print("--------------------做中雲填高雲、低雲填中雲")
print(df.head(20))

df[['low','mid','hig','cloud']] = df[['low','mid','hig','cloud']].interpolate()
print("--------------------先做一次線性")
print(df.head(20))

# 將date_time欄位設定為索引
df.set_index('fst_time', inplace=True)

# 進行重取樣，將十分鐘一筆資料重取樣為5分鐘一筆
df = df.resample('5T').first().reset_index()
print("--------------------重取樣為5分鐘一筆")
print(df.head(20))

df['lon'].fillna('120.560722', inplace=True)
df['lat'].fillna('24.066583', inplace=True)

df[['low','mid','hig','cloud']] = df[['low','mid','hig','cloud']].interpolate()
print("--------------------做第二次線性")
print(df.head(20))

# 將date_time轉換為datetime格式
df['fst_time'] = pd.to_datetime(df['fst_time'])

# 選擇時間等於15、30、45或00分的資料
df = df[df['fst_time'].dt.minute.isin([15, 30, 45, 0])]
print("--------------------只取15, 30, 45, 0的時間")
print(df.head(20))


# 輸出df2
df.to_csv(f'./cloud_cover/10m_to_15m_hig_colud_5_16.csv', index=None)

--------------------初始
              date_time            fst_time         lon        lat      low  \
0   2022-02-01 00:50:00 2022-02-01 00:50:00  120.560722  24.066583  70.7373   
1   2022-02-01 00:50:00 2022-02-01 01:00:00  120.560722  24.066583  63.2563   
2   2022-02-01 00:50:00 2022-02-01 01:10:00  120.560722  24.066583  55.5953   
3   2022-02-01 00:50:00 2022-02-01 01:20:00  120.560722  24.066583  48.3026   
4   2022-02-01 00:50:00 2022-02-01 01:30:00  120.560722  24.066583  41.9924   
5   2022-02-01 00:50:00 2022-02-01 01:40:00  120.560722  24.066583  37.1472   
6   2022-02-01 00:50:00 2022-02-01 01:50:00  120.560722  24.066583  33.9290   
7   2022-02-01 00:50:00 2022-02-01 02:00:00  120.560722  24.066583  32.1273   
8   2022-02-01 00:50:00 2022-02-01 02:10:00  120.560722  24.066583  31.2860   
9   2022-02-01 01:50:00 2022-02-01 02:20:00  120.560722  24.066583  46.6928   
10  2022-02-01 00:50:00 2022-02-01 02:30:00  120.560722  24.066583  30.4214   
11  2022-02-01 00:50:00 2022-

In [2]:
#history_15的csv檔
df_X = pd.read_csv(f'./dataset/solar_汙水廠_newbig_sort(history_15m).csv')
dfXlen = len(df_X)

#篩選時間大於等於2022/2/1小於等於2022/10/31(因為雲資料集只有該區間時間資料)
dfXlen_result = df_X[(df_X['TIME_TO_INTERVAL'] >= '2022-02-01 00:00:00') 
                & (df_X['TIME_TO_INTERVAL'] < '2022-11-01 00:00:00')]

#將原有索引替換為從0開始的新索引
dfXlen_result = dfXlen_result.reset_index(drop=True).reset_index(drop=True)
df = df.reset_index(drop=True)

#合併history_15的csv檔
df_all = pd.concat([dfXlen_result,df],axis=1)

#存成csv
df_all.to_csv(f'./dataset/marge_10m_to_15m_hig_colud.csv', index=None)

In [3]:
# 讀取資料
# df = pd.read_csv(f'./dataset/tt).csv')
# df[28863:28900]